In [213]:
import theano.tensor as T
import theano
import numpy as np
from scipy.spatial.distance import pdist, squareform
import random
import time
import pandas as pd

'''
    Sample code to reproduce our results for the Bayesian neural network example.
    Our settings are almost the same as Hernandez-Lobato and Adams (ICML15) https://jmhldotorg.files.wordpress.com/2015/05/pbp-icml2015.pdf
    Our implementation is also based on their Python code.
    
    p(y | W, X, \gamma) = \prod_i^N  N(y_i | f(x_i; W), \gamma^{-1})
    p(W | \lambda) = \prod_i N(w_i | 0, \lambda^{-1})
    p(\gamma) = Gamma(\gamma | a0, b0)
    p(\lambda) = Gamma(\lambda | a0, b0)
    
    The posterior distribution is as follows:
    p(W, \gamma, \lambda) = p(y | W, X, \gamma) p(W | \lanmbda) p(\gamma) p(\lambda) 
    To avoid negative values of \gamma and \lambda, we update loggamma and loglambda instead.
    
    Copyright (c) 2016,  Qiang Liu & Dilin Wang
    All rights reserved.
'''


class svgd_bayesnn:

    '''
        We define a one-hidden-layer-neural-network specifically. We leave extension of deep neural network as our future work.
        
        Input
            -- X_train: training dataset, features
            -- y_train: training labels
            -- batch_size: sub-sampling batch size
            -- max_iter: maximum iterations for the training procedure
            -- M: number of particles are used to fit the posterior distribution
            -- n_hidden: number of hidden units
            -- a0, b0: hyper-parameters of Gamma distribution
            -- master_stepsize, auto_corr: parameters of adgrad
    '''
    def __init__(self, X_train, y_train,  batch_size = 100, max_iter = 100, M = 100, n_hidden = 50, a0 = 1, b0 = 0.1, master_stepsize = 1e-3, auto_corr = 0.9,beta=-0.5):
        self.n_hidden = n_hidden
        self.d = X_train.shape[1]   # number of data, dimension 
        self.M = M
        self.beta=beta
        self.auto_corr=auto_corr

        self.master_stepsize=master_stepsize
        num_vars = self.d * n_hidden + n_hidden * 2 + 3  # w1: d*n_hidden; b1: n_hidden; w2 = n_hidden; b2 = 1; 2 variances
        self.theta = np.zeros([self.M, num_vars])  # particles, will be initialized later
        
        '''
            We keep the last 10% (maximum 500) of training data points for model developing
        '''
        size_dev = min(int(np.round(0.1 * X_train.shape[0])), 500)
        X_dev, y_dev = X_train[-size_dev:], y_train[-size_dev:]
        X_train, y_train = X_train[:-size_dev], y_train[:-size_dev]

        '''
            The data sets are normalized so that the input features and the targets have zero mean and unit variance
        '''
        self.std_X_train = np.std(X_train, 0)
        self.std_X_train[ self.std_X_train == 0 ] = 1
        self.mean_X_train = np.mean(X_train, 0)
                
        self.mean_y_train = np.mean(y_train)
        self.std_y_train = np.std(y_train)
        
        '''
            Theano symbolic variables
            Define the neural network here
        '''
        X = T.matrix('X') # Feature matrix
        y = T.vector('y') # labels
        
        w_1 = T.matrix('w_1') # weights between input layer and hidden layer
        b_1 = T.vector('b_1') # bias vector of hidden layer
        w_2 = T.vector('w_2') # weights between hidden layer and output layer
        b_2 = T.scalar('b_2') # bias of output
        
        N = T.scalar('N') # number of observations
        
        log_gamma = T.scalar('log_gamma')   # variances related parameters
        log_lambda = T.scalar('log_lambda')
        
        ###
        prediction = T.dot(T.nnet.relu(T.dot(X, w_1)+b_1), w_2) + b_2
        
        ''' define the log posterior distribution '''
        log_lik_data = -0.5 * X.shape[0] * (T.log(2*np.pi) - log_gamma) - (T.exp(log_gamma)/2) * T.sum(T.power(prediction - y, 2))
        log_prior_data = (a0 - 1) * log_gamma - b0 * T.exp(log_gamma) + log_gamma
        log_prior_w = -0.5 * (num_vars-2) * (T.log(2*np.pi)-log_lambda) - (T.exp(log_lambda)/2)*((w_1**2).sum() + (w_2**2).sum() + (b_1**2).sum() + b_2**2)  \
                       + (a0-1) * log_lambda - b0 * T.exp(log_lambda) + log_lambda
        
        # sub-sampling mini-batches of data, where (X, y) is the batch data, and N is the number of whole observations
        log_posterior = (log_lik_data * N / X.shape[0] + log_prior_data + log_prior_w)
        dw_1, db_1, dw_2, db_2, d_log_gamma, d_log_lambda = T.grad(log_posterior, [w_1, b_1, w_2, b_2, log_gamma, log_lambda])
        
        # automatic gradient
        self.logp_gradient = theano.function(
             inputs = [X, y, w_1, b_1, w_2, b_2, log_gamma, log_lambda, N],
             outputs = [dw_1, db_1, dw_2, db_2, d_log_gamma, d_log_lambda]
        )
        
        # prediction function
        self.nn_predict = theano.function(inputs = [X, w_1, b_1, w_2, b_2], outputs = prediction)
        
        '''
            Training with SVGD(beta=0) and Beta SVGD
        '''
        # normalization
        X_train, y_train = self.normalization(X_train, y_train)
        N0 = X_train.shape[0]  # number of observations
        
        ''' initializing all particles '''
        #np.random.seed(300)
        for i in range(self.M):
            w1, b1, w2, b2, loggamma, loglambda = self.init_weights(a0, b0)
            # use better initialization for gamma
            ridx = np.random.choice(range(X_train.shape[0]), \
                                           np.min([X_train.shape[0], 1000]), replace = False)
            y_hat = self.nn_predict(X_train[ridx,:], w1, b1, w2, b2)
            loggamma = -np.log(np.mean(np.power(y_hat - y_train[ridx], 2)))
            self.theta[i,:] = self.pack_weights(w1, b1, w2, b2, loggamma, loglambda)



        '''self variables'''    
        self.X_train = X_train
        self.y_train=y_train
        self.N0=N0
        self.num_vars=num_vars
        self.max_iter=max_iter
        self.batch_size=batch_size
        self.X_dev=X_dev
        self.y_dev=y_dev
        self.grad_theta = np.zeros([self.M, self.num_vars])  # gradient 
        omega = 1/self.M*np.ones(self.M)
        d_kernal_xj_xi = np.zeros([self.M,self.M])

        bandwidth = np.sqrt(self.theta.shape(1))
        for iter in range(self.max_iter):
            # sub-sampling
            batch = [ i % self.N0 for i in range(iter * self.batch_size, (iter + 1) * self.batch_size) ]
            for i in range(self.M):
                w1, b1, w2, b2, loggamma, loglambda = self.unpack_weights(self.theta[i,:])
                dw1, db1, dw2, db2, dloggamma, dloglambda = self.logp_gradient(self.X_train[batch,:], self.y_train[batch], w1, b1, w2, b2, loggamma, loglambda, self.N0)
                self.grad_theta[i,:] = self.pack_weights(dw1, db1, dw2, db2, dloggamma, dloglambda)
                
            # calculating the kernel matrix
            if self.beta==0:
                kxy, dxkxy = self.svgd_kernel(h=bandwidth)  
                self.grad_theta = (np.matmul(kxy, self.grad_theta) + dxkxy) / self.M   

            else:
            
                kxy, dxkxy, ddkxy, hh = self.betasvgd_kernel(h=bandwidth) 
    
                for i_index in range(self.M):
                    for j_index in range(self.M):
                        d_kernal_xj_xi[i_index][j_index] = np.matmul(self.theta[i_index]-self.theta[j_index],(self.grad_theta[i_index]-self.grad_theta[j_index]).transpose())
                d_kernal_xj_xi = np.multiply((d_kernal_xj_xi)*(2/hh),kxy)

                SteinMatrix = (np.multiply(kxy,np.matmul(self.grad_theta,self.grad_theta.transpose()))+ddkxy+d_kernal_xj_xi) /10**4  #rescale the Stein Matrix by a factor 10**4
            
                omega = self.mirror_descent(SteinMatrix,50,0.5,omega,0.2) 

                self.grad_theta = (np.matmul(kxy, self.grad_theta) + dxkxy) / self.M   
                self.grad_theta = np.matmul(np.diag(np.maximum(self.M*omega,0.25)**(self.beta)),self.grad_theta)

            self.theta = self.theta + self.master_stepsize * self.grad_theta 


        '''
            Model selection by using a development set

        '''

        X_dev = self.normalization(self.X_dev) 
        for i in range(self.M):
            w1, b1, w2, b2, loggamma, loglambda = self.unpack_weights(self.theta[i, :])
            pred_y_dev = self.nn_predict(X_dev, w1, b1, w2, b2) * self.std_y_train + self.mean_y_train
            # likelihood
            def f_log_lik(loggamma): return np.sum(  np.log(np.sqrt(np.exp(loggamma)) /np.sqrt(2*np.pi) * np.exp( -1 * (np.power(pred_y_dev - self.y_dev, 2) / 2) * np.exp(loggamma) )) )
            # The higher probability is better    
            lik1 = f_log_lik(loggamma)
            # one heuristic setting
            loggamma = -np.log(np.mean(np.power(pred_y_dev -self.y_dev, 2)))
            lik2 = f_log_lik(loggamma)
            if lik2 > lik1:
                self.theta[i,-2] = loggamma  # update loggamma


    def normalization(self, X, y = None):
        X = (X - np.full(X.shape, self.mean_X_train)) / \
            np.full(X.shape, self.std_X_train)
            
        if y is not None:
            y = (y - self.mean_y_train) / self.std_y_train
            return (X, y)  
        else:
            return X
    
    '''
        Initialize all particles
    '''
    def init_weights(self, a0, b0):
        w1 = 1.0 / np.sqrt(self.d + 1) * np.random.randn(self.d, self.n_hidden)
        b1 = np.zeros((self.n_hidden,))
        w2 = 1.0 / np.sqrt(self.n_hidden + 1) * np.random.randn(self.n_hidden)
        b2 = 0.
        loggamma = np.log(np.random.gamma(a0, b0))
        loglambda = np.log(np.random.gamma(a0, b0))
        return (w1, b1, w2, b2, loggamma, loglambda)
    
    '''
        Calculate kernel matrix and its gradient: K, \nabla_x k
    ''' 
    def svgd_kernel(self, h = -1):
        sq_dist = pdist(self.theta)
        pairwise_dists = squareform(sq_dist)**2
        if h < 0: # if h < 0, using median trick
            h = np.median(pairwise_dists)  
            h = np.sqrt(0.5 * h / np.log(self.theta.shape[0]+1))

        # compute the rbf kernel
        
        Kxy = np.exp( -pairwise_dists / h**2 / 2)

        dxkxy = -np.matmul(Kxy, self.theta)
        sumkxy = np.sum(Kxy, axis=1)
        for i in range(self.theta.shape[1]):
            dxkxy[:, i] = dxkxy[:,i] + np.multiply(self.theta[:,i],sumkxy)
        dxkxy = dxkxy / (h**2)
        return (Kxy, dxkxy)

    def betasvgd_kernel(self, h = -1):
        d = self.theta.shape[1]
        sq_dist = pdist(self.theta)
        pairwise_dists = squareform(sq_dist)**2
        if h < 0: # if h < 0, using median trick
            h = np.median(pairwise_dists)  
            h = np.sqrt(0.5 * h / np.log(self.theta.shape[0]+1))

        Kxy = np.exp( -pairwise_dists / h**2 / 2) # the bandwith for the reproducing kernal here is 2*h**2

        dxkxy = -np.matmul(Kxy, self.theta)
        sumkxy = np.sum(Kxy, axis=1)
        for i in range(self.theta.shape[1]):
            dxkxy[:, i] = dxkxy[:,i] + np.multiply(self.theta[:,i],sumkxy)
        dxkxy = dxkxy / (h**2)
        
        ddkxy=np.zeros([self.theta.shape[0],self.theta.shape[0]])
        ddkxy= np.multiply(Kxy,d/h**2-pairwise_dists/h**4)
        return (Kxy, dxkxy, ddkxy, 2*h**2)
    
    def mirror_descent(self,A,iter,stepsize,omega,alpha):
        omega_m = omega
        n = omega.shape[0]

        for j in range(iter):
            omega_m = alpha*omega_m + np.matmul(A,omega)
            for i in range(n):
                omega[i]=omega[i]*np.exp(-stepsize*omega_m[i])
            omega = (1/omega.sum())*omega 

        return omega 
    
    
    '''
        Pack all parameters in our model
    '''    
    def pack_weights(self, w1, b1, w2, b2, loggamma, loglambda):
        params = np.concatenate([w1.flatten(), b1, w2, [b2], [loggamma],[loglambda]])
        return params
    
    '''
        Unpack all parameters in our model
    '''
    def unpack_weights(self, z):
        w = z
        w1 = np.reshape(w[:self.d*self.n_hidden], [self.d, self.n_hidden])
        b1 = w[self.d*self.n_hidden:(self.d+1)*self.n_hidden]
    
        w = w[(self.d+1)*self.n_hidden:]
        w2, b2 = w[:self.n_hidden], w[-3] 
        
        # the last two parameters are log variance
        loggamma, loglambda= w[-2], w[-1]
        
        return (w1, b1, w2, b2, loggamma, loglambda)

    
    '''
        Evaluating testing rmse and log-likelihood, which is the same as in PBP 
        Input:
            -- X_test: unnormalized testing feature set
            -- y_test: unnormalized testing labels
    '''
    def evaluation(self, X_test, y_test):
        # normalization
        X_test = self.normalization(X_test)
        
        # average over the output
        pred_y_test = np.zeros([self.M, len(y_test)])
        prob = np.zeros([self.M, len(y_test)])
        
        '''
            Since we have M particles, we use a Bayesian view to calculate rmse and log-likelihood
        '''
        for i in range(self.M):
            w1, b1, w2, b2, loggamma, loglambda = self.unpack_weights(self.theta[i, :])
            pred_y_test[i, :] = self.nn_predict(X_test, w1, b1, w2, b2) * self.std_y_train + self.mean_y_train
            prob[i, :] = np.sqrt(np.exp(loggamma)) /np.sqrt(2*np.pi) * np.exp( -1 * (np.power(pred_y_test[i, :] - y_test, 2) / 2) * np.exp(loggamma) )
        pred = np.mean(pred_y_test, axis=0)
        
        # evaluation
        svgd_rmse = np.sqrt(np.mean((pred - y_test)**2))
        svgd_ll = np.mean(np.log(np.mean(prob, axis = 0)))
        
        return (svgd_rmse, svgd_ll)



In [214]:
seeds=1 #we set it as 1 in the experiments
testn=1
betan =-0.5
batch_size, n_hidden, max_iter,master_stepsize= 100, 50, 1000,1e-4 

In [ ]:
np.random.seed(seeds)
random.seed(seeds)
''' load data file '''
#data = np.loadtxt('boston_housing')
data = np.loadtxt('seeds.txt')
#data = np.loadtxt('yacht.txt')
# Please make sure that the last column is the label and the other columns are features
X_input = data[ :, range(data.shape[ 1 ] - 1) ]
y_input = data[ :, data.shape[ 1 ] - 1 ]

''' build the training and testing data set'''
train_ratio = 0.9 # We create the train and test sets with 90% and 10% of the data
permutation = np.arange(X_input.shape[0])
random.shuffle(permutation) 

size_train = int(np.round(X_input.shape[ 0 ] * train_ratio))
index_train = permutation[ 0 : size_train]
index_test = permutation[ size_train : ]

X_train, y_train = X_input[ index_train, : ], y_input[ index_train ]
X_test, y_test = X_input[ index_test, : ], y_input[ index_test ]


test_out1=np.zeros([10,2])
for i in range(testn):
    svgd = svgd_bayesnn(X_train, y_train, batch_size = batch_size, n_hidden = n_hidden, max_iter = max_iter,master_stepsize=master_stepsize,beta=0)
    test_out1[i]=svgd.evaluation(X_test,y_test)
print(test_out1)


In [ ]:
np.random.seed(seeds)
random.seed(seeds)
''' load data file '''
#data = np.loadtxt('boston_housing')
data = np.loadtxt('seeds.txt')
#data = np.loadtxt('yacht.txt')
# Please make sure that the last column is the label and the other columns are features
X_input = data[ :, range(data.shape[ 1 ] - 1) ]
y_input = data[ :, data.shape[ 1 ] - 1 ]

''' build the training and testing data set'''
train_ratio = 0.9 # We create the train and test sets with 90% and 10% of the data
permutation = np.arange(X_input.shape[0])
random.shuffle(permutation) 

size_train = int(np.round(X_input.shape[ 0 ] * train_ratio))
index_train = permutation[ 0 : size_train]
index_test = permutation[ size_train : ]

X_train, y_train = X_input[ index_train, : ], y_input[ index_train ]
X_test, y_test = X_input[ index_test, : ], y_input[ index_test ]


test_out2=np.zeros([10,2])
for i in range(testn):
    svgd = svgd_bayesnn(X_train, y_train, batch_size = batch_size, n_hidden = n_hidden, max_iter = max_iter,master_stepsize=master_stepsize,beta=betan)
    test_out2[i]=svgd.evaluation(X_test,y_test)
print(test_out2)

In [ ]:
np.random.seed(seeds)
random.seed(seeds)
''' load data file '''
data = np.loadtxt('boston_housing')
#data = np.loadtxt('seeds.txt')
#data = np.loadtxt('yacht.txt')
# Please make sure that the last column is the label and the other columns are features
X_input = data[ :, range(data.shape[ 1 ] - 1) ]
y_input = data[ :, data.shape[ 1 ] - 1 ]

''' build the training and testing data set'''
train_ratio = 0.9 # We create the train and test sets with 90% and 10% of the data
permutation = np.arange(X_input.shape[0])
random.shuffle(permutation) 

size_train = int(np.round(X_input.shape[ 0 ] * train_ratio))
index_train = permutation[ 0 : size_train]
index_test = permutation[ size_train : ]

X_train, y_train = X_input[ index_train, : ], y_input[ index_train ]
X_test, y_test = X_input[ index_test, : ], y_input[ index_test ]


test_out3=np.zeros([10,2])
for i in range(testn):
    svgd = svgd_bayesnn(X_train, y_train, batch_size = batch_size, n_hidden = n_hidden, max_iter = max_iter,master_stepsize=master_stepsize,beta=0)
    test_out3[i]=svgd.evaluation(X_test,y_test)
print(test_out3)

In [ ]:
np.random.seed(seeds)
random.seed(seeds)
''' load data file '''
data = np.loadtxt('boston_housing')
#data = np.loadtxt('seeds.txt')
#data = np.loadtxt('yacht.txt')
# Please make sure that the last column is the label and the other columns are features
X_input = data[ :, range(data.shape[ 1 ] - 1) ]
y_input = data[ :, data.shape[ 1 ] - 1 ]

''' build the training and testing data set'''
train_ratio = 0.9 # We create the train and test sets with 90% and 10% of the data
permutation = np.arange(X_input.shape[0])
random.shuffle(permutation) 

size_train = int(np.round(X_input.shape[ 0 ] * train_ratio))
index_train = permutation[ 0 : size_train]
index_test = permutation[ size_train : ]

X_train, y_train = X_input[ index_train, : ], y_input[ index_train ]
X_test, y_test = X_input[ index_test, : ], y_input[ index_test ]


test_out4=np.zeros([10,2])
for i in range(testn):
    svgd = svgd_bayesnn(X_train, y_train, batch_size = batch_size, n_hidden = n_hidden, max_iter = max_iter,master_stepsize=master_stepsize,beta=betan)
    test_out4[i]=svgd.evaluation(X_test,y_test)
print(test_out4)

In [ ]:
np.random.seed(seeds)
random.seed(seeds)
''' load data file '''
#data = np.loadtxt('boston_housing')
#data = np.loadtxt('seeds.txt')
data = np.loadtxt('yacht.txt')
# Please make sure that the last column is the label and the other columns are features
X_input = data[ :, range(data.shape[ 1 ] - 1) ]
y_input = data[ :, data.shape[ 1 ] - 1 ]

''' build the training and testing data set'''
train_ratio = 0.9 # We create the train and test sets with 90% and 10% of the data
permutation = np.arange(X_input.shape[0])
random.shuffle(permutation) 

size_train = int(np.round(X_input.shape[ 0 ] * train_ratio))
index_train = permutation[ 0 : size_train]
index_test = permutation[ size_train : ]

X_train, y_train = X_input[ index_train, : ], y_input[ index_train ]
X_test, y_test = X_input[ index_test, : ], y_input[ index_test ]


test_out5=np.zeros([10,2])
for i in range(testn):
    svgd = svgd_bayesnn(X_train, y_train, batch_size = batch_size, n_hidden = n_hidden, max_iter = max_iter,master_stepsize=master_stepsize,beta=0)
    test_out5[i]=svgd.evaluation(X_test,y_test)
print(test_out5)

In [ ]:
np.random.seed(seeds)
random.seed(seeds)
''' load data file '''
#data = np.loadtxt('boston_housing')
#data = np.loadtxt('seeds.txt')
data = np.loadtxt('yacht.txt')
# Please make sure that the last column is the label and the other columns are features
X_input = data[ :, range(data.shape[ 1 ] - 1) ]
y_input = data[ :, data.shape[ 1 ] - 1 ]

''' build the training and testing data set'''
train_ratio = 0.9 # We create the train and test sets with 90% and 10% of the data
permutation = np.arange(X_input.shape[0])
random.shuffle(permutation) 

size_train = int(np.round(X_input.shape[ 0 ] * train_ratio))
index_train = permutation[ 0 : size_train]
index_test = permutation[ size_train : ]

X_train, y_train = X_input[ index_train, : ], y_input[ index_train ]
X_test, y_test = X_input[ index_test, : ], y_input[ index_test ]


test_out6=np.zeros([10,2])
for i in range(testn):
    svgd = svgd_bayesnn(X_train, y_train, batch_size = batch_size, n_hidden = n_hidden, max_iter = max_iter,master_stepsize=master_stepsize,beta=betan)
    test_out6[i]=svgd.evaluation(X_test,y_test)
print(test_out6)